In [1]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import dot

%load_ext watermark
%watermark --iversions

arviz: 0.11.4
numpy: 1.22.3
pymc : 4.0.0b4



# Rats 

I'm just going to do ratsignorable2.odc for now since it is relevant for HW6. Eventually I'll add the other examples.

Adapted from [Codes for Unit 8: ratsignorable2.odc](https://www2.isye.gatech.edu/isye6420/supporting.html).

Associated lecture video: [Unit 8 Lesson 2](https://www.youtube.com/watch?v=T5vkLsIs3f8&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=83).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/rats.txt).

We had a previous example about [Dugongs](https://areding.github.io/6420-pymc/Unit6-dugongs.html) that dealt with missing data in the observed data (y values). This example shows how to deal with missing data in the input data (x). It's still pretty easy. You could look at it like creating another likelihood in the model, a very simple one where the observed data is x, and you use a single distribution to fill in the missing values (see ```x_imputed``` in the model below).

For now I'm leaving the variable names the same as the BUGS example. Might go back and make them more descriptive later.

There are some differences with my version:

1. My gamma priors on tau are more informative. This is because PyMC was having some computational issues with the Gamma(.001, .001) priors the professor used. I don't know if it's because of the sampling algorithm or if it's a problem with the new computational backend. I will look into it more later, for now I just want to get these examples up.

2. I imputed the x values with a more informative prior for similar reasons. That Uniform(0, 500) prior seems kind of crazy to me, and I wanted to rule out more computational issues.

3. I got rid of the separate definition of the intercept as alpha, it is now beta[0].

The priors are besides the point anyways, I just wanted to show how to impute x values for HW6!

In [2]:
# note that I added a 1 to the first value for x, this is for the intercept beta[0]
x = [1.0, 8.0, 15.0, 22.0, np.nan, 36.0]
y = np.loadtxt("./data/rats.txt")
y.shape

(30, 5)

In [3]:
# create masked data
y = y.copy()
y = np.nan_to_num(y, nan=-1)
y = np.ma.masked_values(y, value=-1)

x = x.copy()
x = np.nan_to_num(x, nan=-1)
x = np.ma.masked_values(x, value=-1)

In [5]:
with pm.Model() as m:
    x_data = pm.Data("x_data", x, mutable=True)
    y_data = pm.Data("y_data", y, mutable=False)

    tau_c = pm.Gamma("tau.c", 3, 0.5)
    beta_c = pm.Normal("beta.c", 0, tau=1e-6)
    beta_tau = pm.Gamma("beta.tau", 3, 0.5)

    beta = pm.Normal("beta", beta_c, tau=0.01, shape=6)

    x_imputed = pm.Normal("x_imputed", mu=20, sigma=10, observed=x_data)

    mu = dot(beta, x_imputed)
    likelihood = pm.Normal("likelihood", mu, tau=0.01, observed=y_data, shape=5)

    trace = pm.sample(
        2000,
        cores=4,
        init="jitter+adapt_diag",
    )

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau.c, beta.c, beta.tau, beta]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 175 seconds.


In [6]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta.c,1.743,5.664,-9.253,12.811,0.081,0.060,4900.0,5289.0,1.0
beta[0],1.768,11.393,-20.379,23.810,0.138,0.118,6833.0,5962.0,1.0
beta[1],1.586,10.776,-19.371,22.694,0.133,0.118,6522.0,5729.0,1.0
beta[2],1.739,9.781,-16.872,20.716,0.126,0.110,6003.0,5529.0,1.0
beta[3],1.626,8.979,-15.908,19.329,0.118,0.104,5830.0,5234.0,1.0
beta[4],1.832,11.710,-21.728,24.133,0.151,0.128,6056.0,5542.0,1.0
beta[5],1.558,6.588,-10.809,14.511,0.100,0.075,4362.0,4721.0,1.0
tau.c,6.047,3.494,0.435,12.739,0.037,0.028,7618.0,4104.0,1.0
beta.tau,6.007,3.554,0.619,12.872,0.037,0.027,7243.0,3680.0,1.0
